# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298317178055                   -0.85    4.9         
  2   -8.300183186176       -2.73       -1.26    1.8    136ms
  3   -8.300432042700       -3.60       -1.87    2.1    142ms
  4   -8.300460506862       -4.55       -2.68    2.0    191ms
  5   -8.300464193523       -5.43       -3.19    2.5    157ms
  6   -8.300464401690       -6.68       -3.32    8.6    261ms
  7   -8.300464501645       -7.00       -3.45    2.5    170ms
  8   -8.300464572382       -7.15       -3.59    1.2    133ms
  9   -8.300464626065       -7.27       -3.81    1.6    170ms
 10   -8.300464640246       -7.85       -4.01    2.0    145ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64204197831                   -0.70    5.4         
  2   -16.67784651255       -1.45       -1.14    1.4    337ms
  3   -16.67921662238       -2.86       -1.88    3.6    404ms
  4   -16.67927936000       -4.20       -2.73    4.6    422ms
  5   -16.67928595283       -5.18       -3.09    3.6    453ms
  6   -16.67928619855       -6.61       -3.51    2.0    333ms
  7   -16.67928621742       -7.72       -4.01    2.2    331ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32535904850                   -0.56    6.8         
  2   -33.33269705579       -2.13       -1.00    1.0    1.23s
  3   -33.33410693093       -2.85       -1.74    3.9    1.55s
  4   -33.33427973275       -3.76       -2.64    4.1    1.47s
  5   -33.33579763958       -2.82       -2.43    8.4    2.14s
  6   -33.33694281048       -2.94       -2.53    4.5    1.51s
  7   -33.33694294239       -6.88       -2.54    1.0    1.19s
  8   -33.33679986551   +   -3.84       -2.33    3.5    1.55s
  9   -33.33675069909   +   -4.31       -2.27    1.0    1.17s
 10   -33.33677291603       -4.65       -2.29    1.0    1.22s
 11   -33.33681768968       -4.35       -2.36    1.0    1.23s
 12   -33.33683110127       -4.87      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298616872916                   -0.85    5.1         
  2   -8.300271379659       -2.78       -1.59    1.0    102ms
  3   -8.300437587034       -3.78       -2.65    2.9    139ms
  4   -8.300418682989   +   -4.72       -2.42    8.8    282ms
  5   -8.300464293120       -4.34       -3.35    1.0    116ms
  6   -8.300464578289       -6.54       -3.76    4.6    206ms
  7   -8.300464642138       -7.19       -4.41    1.4    144ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31594196334                   -0.56    6.1         
  2   -33.30867836587   +   -2.14       -1.27    1.0    1.54s
  3   -17.21320644893   +    1.21       -0.46    5.9    2.22s
  4   -33.30263662055        1.21       -1.68    6.5    2.32s
  5   -33.15632756705   +   -0.83       -1.33    3.4    1.64s
  6   -32.44780812333   +   -0.15       -1.09    5.0    1.84s
  7   -33.28740095315       -0.08       -1.64    4.2    1.71s
  8   -33.33541803904       -1.32       -2.26    2.2    1.21s
  9   -33.33667798655       -2.90       -2.55    3.2    1.64s
 10   -33.33597335747   +   -3.15       -2.43    2.9    1.57s
 11   -33.33692218743       -3.02       -3.02    2.2    1.37s
 12   -33.33694072894       -4.73      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.